In [64]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [65]:
with open('intents.json') as file:
    data = json.load(file)

In [66]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [67]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [68]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [69]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_7   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_21 (Dense)            (None, 16)                272       
                                                                 
 dense_22 (Dense)            (None, 16)                272       
                                                                 
 dense_23 (Dense)            (None, 26)                442       
                                                                 
Total params: 16,986
Trainable params: 16,986
Non-trainable params: 0
__________________________________________________

In [71]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)



Epoch 1/550
3/3 [==============================] - 0s 2ms/step - loss: 3.2567 - accuracy: 0.0526
Epoch 2/550
3/3 [==============================] - 0s 2ms/step - loss: 3.2539 - accuracy: 0.0526
Epoch 3/550
3/3 [==============================] - 0s 2ms/step - loss: 3.2520 - accuracy: 0.0526
Epoch 4/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2499 - accuracy: 0.0526
Epoch 5/550
3/3 [==============================] - 0s 2ms/step - loss: 3.2469 - accuracy: 0.0526
Epoch 6/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2446 - accuracy: 0.0526
Epoch 7/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2421 - accuracy: 0.0658
Epoch 8/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2394 - accuracy: 0.1316
Epoch 9/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2366 - accuracy: 0.1316
Epoch 10/550
3/3 [==============================] - 0s 3ms/step - loss: 3.2339 - accuracy: 0.1316
Epoch 11/550
3/3 [===========

In [72]:
import pickle
# saving model
model.save("chat_model")




# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets
